In [1]:
from sklearn.model_selection import StratifiedShuffleSplit, StratifiedKFold
from sklearn import metrics
from contextlib import redirect_stdout
from itertools import compress
import scipy.sparse as sparse
import lightgbm as lgb
import pandas as pd
import numpy as np
import tqdm
import os
import gc
import sys
sys.path.append('../../../code/pipeline/')
sys.path.append('../../../code/utils/')
sys.path.append('../../../code/')
import data_pipeline as dp
import data_utils as du
import perf_utils as pu
import eval_utils as eu
import io_utils as iu
import config

In [2]:
# ================
# Data Preparation
# ================
# defined feature pairs to load cross product transformation
# pairs = [("aid", "age"), ("aid", "education"), ("aid", "consumptionAbility"), ("aid", "LBS")]
pairs = [('productId', 'LBS'),
         ('advertiserId', 'interest1'),
         ('aid', 'interest2'),
         ('creativeSize', 'interest2'), 
         ('campaignId', 'interest4'),  # whether to keep it? 
         ('aid', 'interest5'),  
         ('productType', 'kw1'),  # 'kw1' looks very overfitting prone, to be decide whether to keep it
         ('productType', 'kw2'),
         ('productType', 'kw3'),
         ('productType', 'topic1'),
         ('aid', 'topic2'),
         ('productType', 'topic2'),
         # ('productType', 'topic3'),  # might help in predicting negative samples
         # ('productType', 'appIdInstall'),  # might help in predicting negative samples
         # ('productType', 'appIdAction'),  # might help in predicting negative samples
         ('aid', 'ct'),
         ('aid', 'os')]

# instantiate data loader
dm = dp.DataManager(config.INPUT_DIR)
bin_loader = dm.build_data("raw", "binary")
cross_bin_loaders = (dp.CrossBinaryDataManager.build_data(ad_feat_name, user_feat_name) 
                     for ad_feat_name, user_feat_name in pairs)
union_loader = dp.DataUnion(bin_loader, *cross_bin_loaders)

with pu.profiler("loading training data"):
    cols_train, X_tv = union_loader.load("train")
    X_tv = sparse.csr_matrix(X_tv)
    gc.collect()

print("Train Data Shape: {}".format(X_tv.shape))
print("Train Column Numbers: {}".format(len(cols_train)))

[03:59:08] Finish loading training data. △M: +6.23GB. △T: 5.9 minutes.
Train Data Shape: (8798814, 3590073)
Train Column Numbers: 3590073


In [3]:
df_train = du.load_raw_data("train")
y = df_train['label'].values.copy()
y = (y + 1) / 2  # -1, 1 -> 0, 1

In [4]:
# n_splits = 3
# sss = StratifiedShuffleSplit(n_splits=n_splits, test_size=1 / 3, random_state=20180505)  # for reproducibility
# split_indices = [(train_index, valid_index) for train_index, valid_index in sss.split(df_train, y)]

n_splits = 5  # use 3 instead of 5 to save time
skf = StratifiedKFold(n_splits=n_splits)
split_indices = [(train_index, valid_index) for train_index, valid_index in skf.split(df_train, y)]

In [6]:
aids = df_train['aid'].values
with pu.profiler("splitting train/valid set"):
    train_index, valid_index = split_indices[0]
    X_train, X_valid = X_tv[train_index, :], X_tv[valid_index, :]
    y_train, y_valid = y[train_index], y[valid_index]
    aids_train, aids_valid = aids[train_index], aids[valid_index]
    assert X_train.shape[0] + X_valid.shape[0] == X_tv.shape[0]
    
    del X_tv
    gc.collect()

print("Training Set Size (Before masking): {}".format(X_train.shape))
print("Validation Set Size (Before masking): {}".format(X_valid.shape))
print()
    
min_df = 3  # there is no such things for v2
with pu.profiler("trimming train/valid set"):    
    mask = (X_train.getnnz(axis=0) >= min_df)  # threshold to be extended
    cols_masked = list(compress(cols_train, mask))
    X_train = X_train[:, mask]
    X_valid = X_valid[:, mask]
    gc.collect()
    
print("Training Set Size (After masking): {}".format(X_train.shape))
print("Validation Set Size (After masking): {}".format(X_valid.shape))

[04:01:47] Finish splitting train/valid set. △M: +181.8MB. △T: 38.3 seconds.
Training Set Size (Before masking): (7039050, 3590073)
Validation Set Size (Before masking): (1759764, 3590073)

[04:04:19] Finish trimming train/valid set. △M: +40.53MB. △T: 2.5 minutes.
Training Set Size (After masking): (7039050, 1791142)
Validation Set Size (After masking): (1759764, 1791142)


In [8]:
with pu.profiler("preparing LightGBM data"):
    # lgb_train = lgb.Dataset(X_train.astype(np.float32), y_train, feature_name=cols_train)  # cause bugs I don't know how to fix
    # lgb_valid = lgb.Dataset(X_valid.astype(np.float32), y_valid, feature_name=cols_train)
    lgb_train = lgb.Dataset(X_train.astype(np.float32), y_train)
    lgb_valid = lgb.Dataset(X_valid.astype(np.float32), y_valid)
    gc.collect()

[04:05:10] Finish preparing LightGBM data. △M: +9.45GB. △T: 9.3 seconds.


In [9]:
log_folder = os.path.join(config.LOG_DIR, 'lgbm/pipeline/0512/')
log_file = 'v1.log'
log_path = os.path.join(log_folder, log_file)
os.makedirs(log_folder, exist_ok=True)

In [11]:
# v2 parameters
# params = {
#     'boosting_type': 'gbdt',
#     'objective': 'binary',
#     'metric': 'auc',
#     'max_depth': 15,
#     'num_leaves': 120,
#     'learning_rate': 0.15,
#     'feature_fraction': 0.9,
#     'bagging_fraction': 0.8,
#     'verbose': 0
# }
# v3 parameters
params = {
    'boosting_type': 'gbdt',
    'objective': 'binary',
    'metric': 'auc',
    'max_depth': 6,
    'num_leaves': 64,
    'learning_rate': 0.1,
    'feature_fraction': 0.5,
    'bagging_fraction': 0.5,
    'verbose': 0
}
num_rounds = 5000
eval_auc = eu.build_lightgbm_online_auc_eval(aids_train, aids_valid)

with iu.DuplicatedLogger(log_path):
    lgbm = lgb.train(params,
                     lgb_train,
                     num_boost_round=num_rounds,
                     valid_sets=[lgb_train, lgb_valid], 
                     valid_names=['train', 'valid1'],
                     feval = eval_auc,
                     early_stopping_rounds=50)

[1]	train's auc: 0.570279	train's online_auc: 0.583275	valid1's auc: 0.570255	valid1's online_auc: 0.583206
Training until validation scores don't improve for 50 rounds.
[2]	train's auc: 0.622847	train's online_auc: 0.622842	valid1's auc: 0.621829	valid1's online_auc: 0.621137
[3]	train's auc: 0.631631	train's online_auc: 0.632271	valid1's auc: 0.630437	valid1's online_auc: 0.631378
[4]	train's auc: 0.646026	train's online_auc: 0.640083	valid1's auc: 0.645401	valid1's online_auc: 0.640113
[5]	train's auc: 0.650872	train's online_auc: 0.64747	valid1's auc: 0.65005	valid1's online_auc: 0.64815
[6]	train's auc: 0.651037	train's online_auc: 0.647926	valid1's auc: 0.650157	valid1's online_auc: 0.648676
[7]	train's auc: 0.652834	train's online_auc: 0.651013	valid1's auc: 0.651913	valid1's online_auc: 0.652333
[8]	train's auc: 0.652544	train's online_auc: 0.651005	valid1's auc: 0.651652	valid1's online_auc: 0.652247
[9]	train's auc: 0.653752	train's online_auc: 0.650823	valid1's auc: 0.65294	

[77]	train's auc: 0.704712	train's online_auc: 0.700067	valid1's auc: 0.701894	valid1's online_auc: 0.697543
[78]	train's auc: 0.705092	train's online_auc: 0.700472	valid1's auc: 0.702225	valid1's online_auc: 0.697961
[79]	train's auc: 0.70526	train's online_auc: 0.700651	valid1's auc: 0.702388	valid1's online_auc: 0.698161
[80]	train's auc: 0.706061	train's online_auc: 0.701428	valid1's auc: 0.703141	valid1's online_auc: 0.698914
[81]	train's auc: 0.706341	train's online_auc: 0.701709	valid1's auc: 0.703376	valid1's online_auc: 0.699134
[82]	train's auc: 0.70659	train's online_auc: 0.701954	valid1's auc: 0.703501	valid1's online_auc: 0.699223
[83]	train's auc: 0.707161	train's online_auc: 0.70259	valid1's auc: 0.704001	valid1's online_auc: 0.699579
[84]	train's auc: 0.707372	train's online_auc: 0.702626	valid1's auc: 0.704181	valid1's online_auc: 0.699534
[85]	train's auc: 0.707672	train's online_auc: 0.702856	valid1's auc: 0.704418	valid1's online_auc: 0.699702
[86]	train's auc: 0.70

[152]	train's auc: 0.720169	train's online_auc: 0.714586	valid1's auc: 0.714812	valid1's online_auc: 0.708979
[153]	train's auc: 0.720265	train's online_auc: 0.714743	valid1's auc: 0.714855	valid1's online_auc: 0.708982
[154]	train's auc: 0.720364	train's online_auc: 0.714794	valid1's auc: 0.714921	valid1's online_auc: 0.709016
[155]	train's auc: 0.720456	train's online_auc: 0.714842	valid1's auc: 0.71501	valid1's online_auc: 0.70905
[156]	train's auc: 0.720508	train's online_auc: 0.714898	valid1's auc: 0.715057	valid1's online_auc: 0.709106
[157]	train's auc: 0.720556	train's online_auc: 0.714989	valid1's auc: 0.715107	valid1's online_auc: 0.709179
[158]	train's auc: 0.720694	train's online_auc: 0.715061	valid1's auc: 0.715199	valid1's online_auc: 0.709177
[159]	train's auc: 0.72078	train's online_auc: 0.715154	valid1's auc: 0.715254	valid1's online_auc: 0.709246
[160]	train's auc: 0.720926	train's online_auc: 0.715254	valid1's auc: 0.715367	valid1's online_auc: 0.709301
[161]	train's

[227]	train's auc: 0.728981	train's online_auc: 0.723913	valid1's auc: 0.721137	valid1's online_auc: 0.714081
[228]	train's auc: 0.729113	train's online_auc: 0.724283	valid1's auc: 0.721194	valid1's online_auc: 0.714215
[229]	train's auc: 0.729196	train's online_auc: 0.724322	valid1's auc: 0.721235	valid1's online_auc: 0.714225
[230]	train's auc: 0.729284	train's online_auc: 0.72465	valid1's auc: 0.721288	valid1's online_auc: 0.714368
[231]	train's auc: 0.729411	train's online_auc: 0.724824	valid1's auc: 0.72136	valid1's online_auc: 0.714501
[232]	train's auc: 0.729521	train's online_auc: 0.724921	valid1's auc: 0.721403	valid1's online_auc: 0.714529
[233]	train's auc: 0.729555	train's online_auc: 0.724965	valid1's auc: 0.721428	valid1's online_auc: 0.714551
[234]	train's auc: 0.729624	train's online_auc: 0.725038	valid1's auc: 0.721476	valid1's online_auc: 0.714564
[235]	train's auc: 0.729671	train's online_auc: 0.725097	valid1's auc: 0.721502	valid1's online_auc: 0.714611
[236]	train'

[302]	train's auc: 0.734828	train's online_auc: 0.731249	valid1's auc: 0.724703	valid1's online_auc: 0.71724
[303]	train's auc: 0.734896	train's online_auc: 0.731323	valid1's auc: 0.724753	valid1's online_auc: 0.717275
[304]	train's auc: 0.735007	train's online_auc: 0.731369	valid1's auc: 0.724814	valid1's online_auc: 0.71729
[305]	train's auc: 0.73505	train's online_auc: 0.731407	valid1's auc: 0.724842	valid1's online_auc: 0.717311
[306]	train's auc: 0.735154	train's online_auc: 0.731531	valid1's auc: 0.724903	valid1's online_auc: 0.717376
[307]	train's auc: 0.735272	train's online_auc: 0.731639	valid1's auc: 0.724967	valid1's online_auc: 0.717432
[308]	train's auc: 0.735287	train's online_auc: 0.731661	valid1's auc: 0.724978	valid1's online_auc: 0.717444
[309]	train's auc: 0.735316	train's online_auc: 0.731695	valid1's auc: 0.724996	valid1's online_auc: 0.717444
[310]	train's auc: 0.735389	train's online_auc: 0.731749	valid1's auc: 0.725038	valid1's online_auc: 0.717489
[311]	train's

...

[1802]	train's auc: 0.779473	train's online_auc: 0.786644	valid1's auc: 0.739152	valid1's online_auc: 0.732112
[1803]	train's auc: 0.779485	train's online_auc: 0.786659	valid1's auc: 0.739155	valid1's online_auc: 0.732119
[1804]	train's auc: 0.779494	train's online_auc: 0.786661	valid1's auc: 0.73915	valid1's online_auc: 0.732119
[1805]	train's auc: 0.779519	train's online_auc: 0.78669	valid1's auc: 0.739147	valid1's online_auc: 0.73211
[1806]	train's auc: 0.779555	train's online_auc: 0.786721	valid1's auc: 0.739152	valid1's online_auc: 0.732115
[1807]	train's auc: 0.779573	train's online_auc: 0.786736	valid1's auc: 0.739169	valid1's online_auc: 0.732123
[1808]	train's auc: 0.779583	train's online_auc: 0.78674	valid1's auc: 0.739171	valid1's online_auc: 0.732124
[1809]	train's auc: 0.779592	train's online_auc: 0.786756	valid1's auc: 0.739171	valid1's online_auc: 0.732124
[1810]	train's auc: 0.779614	train's online_auc: 0.786771	valid1's auc: 0.739172	valid1's online_auc: 0.732124
[1811

[1877]	train's auc: 0.780846	train's online_auc: 0.788422	valid1's auc: 0.739362	valid1's online_auc: 0.732311
[1878]	train's auc: 0.780867	train's online_auc: 0.788442	valid1's auc: 0.739362	valid1's online_auc: 0.732308
[1879]	train's auc: 0.780879	train's online_auc: 0.788457	valid1's auc: 0.739361	valid1's online_auc: 0.732312
[1880]	train's auc: 0.780909	train's online_auc: 0.788476	valid1's auc: 0.739358	valid1's online_auc: 0.7323
[1881]	train's auc: 0.780928	train's online_auc: 0.788488	valid1's auc: 0.739359	valid1's online_auc: 0.732304
[1882]	train's auc: 0.780942	train's online_auc: 0.788507	valid1's auc: 0.739363	valid1's online_auc: 0.732304
[1883]	train's auc: 0.780951	train's online_auc: 0.788517	valid1's auc: 0.739363	valid1's online_auc: 0.732304
[1884]	train's auc: 0.780963	train's online_auc: 0.788532	valid1's auc: 0.73937	valid1's online_auc: 0.732306
[1885]	train's auc: 0.780967	train's online_auc: 0.788539	valid1's auc: 0.739368	valid1's online_auc: 0.732302
[188

[1952]	train's auc: 0.782119	train's online_auc: 0.79012	valid1's auc: 0.739482	valid1's online_auc: 0.732387
[1953]	train's auc: 0.782131	train's online_auc: 0.790137	valid1's auc: 0.739481	valid1's online_auc: 0.732386
[1954]	train's auc: 0.78214	train's online_auc: 0.790143	valid1's auc: 0.739483	valid1's online_auc: 0.732387
[1955]	train's auc: 0.782152	train's online_auc: 0.790151	valid1's auc: 0.739481	valid1's online_auc: 0.732385
[1956]	train's auc: 0.782168	train's online_auc: 0.790154	valid1's auc: 0.739484	valid1's online_auc: 0.732386
[1957]	train's auc: 0.782176	train's online_auc: 0.790161	valid1's auc: 0.739484	valid1's online_auc: 0.732384
[1958]	train's auc: 0.782184	train's online_auc: 0.790164	valid1's auc: 0.739485	valid1's online_auc: 0.732391
[1959]	train's auc: 0.78221	train's online_auc: 0.790195	valid1's auc: 0.739489	valid1's online_auc: 0.732386
[1960]	train's auc: 0.782232	train's online_auc: 0.790202	valid1's auc: 0.739491	valid1's online_auc: 0.732384
[196

[2027]	train's auc: 0.783479	train's online_auc: 0.791651	valid1's auc: 0.739649	valid1's online_auc: 0.732601
[2028]	train's auc: 0.783504	train's online_auc: 0.791692	valid1's auc: 0.739654	valid1's online_auc: 0.732613
[2029]	train's auc: 0.783532	train's online_auc: 0.791704	valid1's auc: 0.739655	valid1's online_auc: 0.732609
[2030]	train's auc: 0.783539	train's online_auc: 0.791711	valid1's auc: 0.739653	valid1's online_auc: 0.732606
[2031]	train's auc: 0.783552	train's online_auc: 0.791741	valid1's auc: 0.739656	valid1's online_auc: 0.732618
[2032]	train's auc: 0.783565	train's online_auc: 0.791781	valid1's auc: 0.739656	valid1's online_auc: 0.73261
[2033]	train's auc: 0.783575	train's online_auc: 0.791797	valid1's auc: 0.739654	valid1's online_auc: 0.732606
[2034]	train's auc: 0.783589	train's online_auc: 0.791805	valid1's auc: 0.739663	valid1's online_auc: 0.732608
[2035]	train's auc: 0.783599	train's online_auc: 0.791821	valid1's auc: 0.739661	valid1's online_auc: 0.732617
[2

[2102]	train's auc: 0.784793	train's online_auc: 0.793056	valid1's auc: 0.739719	valid1's online_auc: 0.732657
[2103]	train's auc: 0.784807	train's online_auc: 0.793072	valid1's auc: 0.73972	valid1's online_auc: 0.732658
[2104]	train's auc: 0.784818	train's online_auc: 0.793078	valid1's auc: 0.739718	valid1's online_auc: 0.732658
[2105]	train's auc: 0.784826	train's online_auc: 0.793082	valid1's auc: 0.73972	valid1's online_auc: 0.732659
[2106]	train's auc: 0.784846	train's online_auc: 0.793097	valid1's auc: 0.739726	valid1's online_auc: 0.732664
[2107]	train's auc: 0.784861	train's online_auc: 0.793111	valid1's auc: 0.739729	valid1's online_auc: 0.732664
[2108]	train's auc: 0.784869	train's online_auc: 0.793118	valid1's auc: 0.73973	valid1's online_auc: 0.732662
[2109]	train's auc: 0.784885	train's online_auc: 0.793124	valid1's auc: 0.739734	valid1's online_auc: 0.732667
[2110]	train's auc: 0.784898	train's online_auc: 0.793137	valid1's auc: 0.739734	valid1's online_auc: 0.732671
[211

In [13]:
# # these will cause LightGBM bug I don't know how to fix. Just skip it
# log_file = 'v2.feature_importance.csv'
# log_path = os.path.join(log_folder, log_file)

# df_feature_importance = pd.DataFrame({"feature": cols_train, "importance": lgbm.feature_importance()})
# df_feature_importance = df_feature_importance.sort_values("importance", ascending=False)
# df_feature_importance.to_csv(log_path, index=False)
# df_feature_importance.head(30)

In [14]:
with pu.profiler("making prediction on validation set"):
    df_valid = df_train.iloc[valid_index]
    proba_valid = lgbm.predict(X_valid.astype(np.float32))

log_file = 'v1.online_auc.csv'
log_path = os.path.join(log_folder, log_file)
df_score = eu.online_auc(df_valid['aid'], y_valid, proba_valid, ret_verbose=True)
df_score.rename(columns={'selector': 'aid'}, inplace=True)
df_score = df_score[['aid', 'auc']]  # sort columns
df_score = df_score.sort_values("auc", ascending=False)
df_score.to_csv(log_path, index=False)

[15:21:58] Finish making prediction on validation set. △M: +520.45MB. △T: 1.3 minutes.


In [15]:
online_auc = df_score['auc'].mean()
simple_auc = metrics.roc_auc_score(y_valid, proba_valid)
print("Online AUC: {:.6f}".format(online_auc))
print("Simple AUC: {:.6f}".format(simple_auc))

Online AUC: 0.732732
Simple AUC: 0.739721


In [16]:
with pu.profiler("cleaning memory"):
    del lgb_train
    del lgb_valid
    del X_train
    del X_valid
    gc.collect()

[15:22:22] Finish cleaning memory. △M: -17.62GB. △T: 8.7 seconds.


In [17]:
with pu.profiler("loading testing data"):
    cols_test, X_test = union_loader.load("test1")
    X_test = sparse.csr_matrix(X_test)
    gc.collect()
    
with pu.profiler("trimming testing set"):    
    X_test = X_test[:, mask]
    gc.collect()

print("Test Data Shape (After masking): {}".format(X_test.shape))
# print("Test Column Numbers: {}".format(len(cols_test)))

[15:23:44] Finish loading testing data. △M: +1.6GB. △T: 1.2 minutes.
[15:24:12] Finish trimming testing set. △M: -1.22GB. △T: 27.6 seconds.
Test Data Shape (After masking): (2265989, 1791142)


In [18]:
df_test = du.load_raw_data("test")
X_test = X_test.astype(np.float32)

with pu.profiler("making prediction on testing set"):
    proba_test = lgbm.predict(X_test)
    assert len(proba_test.shape) == 1
    assert proba_test.shape[0] == df_test.shape[0]

[15:25:56] Finish making prediction on testing set. △M: -5.14GB. △T: 1.5 minutes.


In [19]:
subm_folder = '../../../subm/lgbm/0512_v1'
subm_file = 'submission.csv'
subm_path = os.path.join(subm_folder, subm_file)
os.makedirs(subm_folder, exist_ok=True)

subm = df_test.copy()
subm["score"] = proba_test
subm.to_csv(subm_path, index=False)